In [5]:
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import os
import asyncio
import ssl
import certifi
import chromadb
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
import mysql.connector
import pythoncom
from datetime import datetime
import win32com.client as win32
import gradio as gr
from dotenv import load_dotenv
from pathlib import Path

os.environ['SSL_CERT_FILE'] = certifi.where()
load_dotenv(override=True)

False

In [ ]:
#===============================================================#
#ESTA VARIABLE SE VERÁ MODIFICADA, YA QUE CAMBIAREMOS DE SERVIDOR
#===============================================================#
MYSQL_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "database": os.getenv("DB_NAME"),
}
print (MYSQL_CONFIG)

{'host': None, 'user': None, 'password': None, 'database': None}


In [ ]:
# Inicialización de clientes globales
cliente_openai = OpenAI()
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")
cliente_chroma = chromadb.PersistentClient(path="db_politicas") #averiguar la manera de almacenar la base vectorial
coleccion = cliente_chroma.get_collection(name="politicas_empresariales")

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
#politicas disponibles
BASE_DIR = Path(__file__).resolve().parent
FILES_DIR = BASE_DIR / "files"

RUTAS_POLITICAS = [
    FILES_DIR / "beca_estudio.pdf",
    FILES_DIR / "centro_recreacion.pdf",
    FILES_DIR / "mutuo_acuerdo.pdf",
]

NOMBRES_POLITICAS = [os.path.basename(ruta) for ruta in RUTAS_POLITICAS]
POLITICAS_CON_DESCRIPCION = {
    "sin_coincidencias": "no se encontro ninguna coincidencia, responde que no conoces la respuesta a su consulta",
    "beca_estudio.pdf": "Contiene información sobre beneficios y becas para estudios superiores para los empleados y sus familias.",
    "centro_recreación.pdf": "Describe las reglas para pertenecer al centro de recreación de la empresa.",
    "mutuo_acuerdo.pdf": "Explica los procedimientos y condiciones para la terminación del contrato laboral de mutuo acuerdo."
}


NameError: name '__file__' is not defined

In [ ]:
#tools orquestador 
@function_tool
def seleccionar_politica_con_llm(pregunta_usuario):
    """
    Usa un LLM para determinar qué política es la más relevante.
    """
    #print(f"Usando LLM para enrutar la pregunta: '{pregunta_usuario}'")

    lista_politicas_formateada = "\n".join(
        [f"- {nombre}: {desc}" for nombre, desc in POLITICAS_CON_DESCRIPCION.items()]
    )

    prompt_enrutador = f"""
    Tu única tarea es actuar como un clasificador de documentos.
    Lee la pregunta del usuario y decide cuál de los siguientes documentos es el más relevante 
    para encontrar la respuesta basándote en su descripción.

    Documentos disponibles:
    {lista_politicas_formateada}

    Pregunta del usuario: "{pregunta_usuario}"

    Responde únicamente con el nombre exacto del archivo del documento más relevante. 
    Si ninguno de los documentos parece relevante para la pregunta, responde con "sin_coincidencias.
    """
    try:
        response = cliente_openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": prompt_enrutador}],
            temperature=0.0
        )
        respuesta_llm = response.choices[0].message.content.strip()
        #print(f"   Respuesta del LLM enrutador: '{respuesta_llm}'")
        
        # Comprobar si el LLM devolvió un nombre de política válido
        for nombre in NOMBRES_POLITICAS:
            if nombre in respuesta_llm:
                #print(f"   Política seleccionada: '{nombre}'")
                return nombre
        
        # Si el LLM devolvió 'N/A' o algo irreconocible, no se encontró una política.
        #print("   El LLM no identificó una política relevante.")
        return "sin_coincidencias" # Devolvemos None explícitamente

    except Exception as e:
        #print(f"   Error en llamada al LLM enrutador: {e}. No se pudo seleccionar política.")
        return "sin_coincidencias"
    
@function_tool
def buscar_contexto_relevante(pregunta, nombre_politica, n_resultados=5):
    """Busca los chunks más relevantes para una pregunta dentro de una política específica."""
    #print(f"Buscando contexto para la pregunta en la política '{nombre_politica}'...")
    embedding_pregunta = embeddings_model.embed_query(pregunta)

    # Filtra la búsqueda para que solo considere la política seleccionada
    resultados = coleccion.query(
        query_embeddings=[embedding_pregunta],
        n_results=n_resultados,
        where={"source": nombre_politica},
        include=["documents"]
    )
    
    documentos_relevantes = resultados['documents'][0] if resultados['documents'] else []
    print(f"   Se encontraron {len(documentos_relevantes)} chunks de contexto relevantes.")
    return documentos_relevantes




In [ ]:
#agente de registro de preguntas
#===============================================================#
#ESTA FUNCIÓN SE VERÁ MODIFICADA, YA QUE CAMBIAREMOS DE SERVIDOR
#===============================================================#

@function_tool
def registrar_pregunta_mysql(pregunta, politica="No especificada", contexto_encontrado=True, respuesta="", notas=""):
    """Registra las preguntas realizadas por el usuario en la base de datos MySQL."""
    try:
        
        conn = mysql.connector.connect(**MYSQL_CONFIG)
        cursor = conn.cursor()
        
        query = """
            INSERT INTO question_agent_ia
            (question, file_consulted, contexts, 
             answer_ia, notes)
            VALUES ( %s, %s, %s, %s, %s)
        """
        valores = (pregunta, politica, contexto_encontrado, respuesta,  notas)
        
        cursor.execute(query, valores)
        conn.commit()
        
        registro_id = cursor.lastrowid
        cursor.close()
        conn.close()
        
        #print(f"Pregunta registrada en MySQL con ID: {registro_id}")
        return {
            "status": "ok", 
            "message": "Pregunta registrada exitosamente",
            "id": registro_id
        }
    except Exception as e:
        #print(f"✗ Error al registrar en MySQL: {e}")
        return {
            "status": "error",
            "message": f"Error al registrar: {str(e)}"
        }

instrucciones_registro = "debes registrar las preguntas de hacen los usuarios a una base mysql"
registro_pregunta = Agent(
    name="registrador de preguntas de usuarios a una base mysql",                       
    instructions=instrucciones_registro, 
    tools=registrar_pregunta_mysql,
    model="gpt-4o-mini",
    handoff_description="ingresa las preguntas del usuario a una base mysql")


In [ ]:
#agente de registro de pregunta desconocidas
#===============================================================#
#ESTA FUNCIÓN SE VERÁ MODIFICADA, YA QUE CAMBIAREMOS DE SERVIDOR
#===============================================================#

@function_tool
def enviar_email_rrhh(asunto, pregunta, rut_usuario="", nombre_usuario="", notas=""):
    """registra y envía un correo electrónico al departamento de RRHH usando Outlook local."""
    try:
        # Registrar en MySQL primero
        conn = mysql.connector.connect(**MYSQL_CONFIG)
        cursor = conn.cursor()
        
        query = """
            INSERT INTO unknown_question
            (pregunta, rut, nombre_usuario, notas)
            VALUES (%s, %s, %s, %s)
        """
        valores = (pregunta, rut_usuario, nombre_usuario, notas)
        
        cursor.execute(query, valores)
        conn.commit()
        
        registro_id = cursor.lastrowid
        cursor.close()
        conn.close()
        
        # print(f"Pregunta registrada en MySQL con ID: {registro_id}")
        # print("Redactando el correo")

        pythoncom.CoInitialize()
        
        try:
            outlook = win32.Dispatch('outlook.application')
            mail = outlook.CreateItem(0)

            emails = os.getenv("EMAIL_RRHH")
            destinatarios = [e.strip() for e in emails.split(",") if e.strip()]

            mail.To = "; ".join(destinatarios)
            mail.Subject = asunto
            cuerpo = f"""Consulta recogida desde el Chatbot de RRHH
De: {nombre_usuario if nombre_usuario else 'Usuario anónimo'}
Rut: {rut_usuario if rut_usuario else 'No proporcionado'}

Pregunta:
{pregunta}

---
Este mensaje fue enviado automáticamente.
Fecha: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}
"""
            mail.Body = cuerpo
            mail.Send()
            
            #print(f"✓ Email enviado a RRHH")
            return {
                "status": "ok",
                "message": "Email enviado exitosamente a RRHH"
            }
        finally:
            pythoncom.CoUninitialize()
            
    except Exception as e:
        #print(f"Error al enviar email: {e}")
        return {
            "status": "error",
            "message": f"No se pudo enviar el email: {str(e)}"
        }
    
instrucciones_registro_desconocido = "debes registrar las preguntas sin respuesta que hacen los usuarios a una base mysql, adicional efecutar el envio del correo informativo a RRHH"
registro_pregunta_desconocida = Agent(
    name="registrador de preguntas desconocidas y envio de correos",                       
    instructions=instrucciones_registro_desconocido, 
    tools=enviar_email_rrhh,
    model="gpt-4o-mini",
    handoff_description="ingresa las preguntas del usuario a una base mysql y envia correos electronicos informativos")



In [ ]:
# agente orquestador 
instrucciones_orquestador = """
Eres un asistente de Recursos Humanos experto de la empresa Cramer. Eres amable y profesional.
Tu misión es responder a las preguntas del usuario.

1.  **Analiza la pregunta del usuario:**
    * Si es un saludo, una despedida o una charla general (ej: "hola", "cómo estás?", "gracias"), responde amablemente sin usar herramientas.
    * Si la pregunta es sobre una política de la empresa (ej: "beneficios de estudio", "club", "finiquito"), procede al paso 2.

2.  **Proceso RAG (Si aplica):**
    a.  Usa la herramienta `seleccionar_politica_con_llm` para identificar el documento relevante.
    b.  **Si la política es 'sin_coincidencias':** Informa al usuario que no tienes información sobre ese tema específico. Procede al Paso 4 (Escalamiento).
    c.  **Si se encuentra una política:** Usa `buscar_contexto_relevante` para obtener los *chunks* de información.

3.  **Formulación de Respuesta (Si hay contexto):**
    a.  **Si `buscar_contexto_relevante` devuelve contexto:** Basa tu respuesta ESTRICTA Y ÚNICAMENTE en ese contexto. Cita los puntos clave.
    b.  **Registro (Handoff):** Una vez formulada tu respuesta, transfiere la pregunta, tu respuesta, y la política usada al agente `registrador_preguntas_usuarios`.
    c.  Entrega la respuesta al usuario.

4.  **Escalamiento (Si no hay respuesta):**
    a.  Si no encontraste la política (paso 2b) o no había contexto (paso 3a), informa al usuario.
    b.  Ofrécete a escalar su consulta a RRHH. Pregúntale si desea hacerlo y si puede proporcionar su nombre y/o RUT.
    c.  **Si el usuario acepta:** Transfiere la pregunta original, y los datos que te dio (nombre/RUT) al agente `registrador_preguntas_desconocidas`.
    d.  **Si el usuario no acepta:** Despídete amablemente.

5.  **Interacción:** Siempre mantén la conversación. Si terminas un flujo, pregunta "¿Tienes alguna otra consulta?".
"""

# Lista de TODAS las herramientas que el orquestador puede usar
herramientas_orquestador = [
    seleccionar_politica_con_llm,
    buscar_contexto_relevante, 
]

handoffs_orquestador = [
    registro_pregunta,
    registro_pregunta_desconocida
]

# CREACIÓN DEL AGENTE ORQUESTADOR
orquestador_agente = Agent(
    name="asistente_rrhh_cramer",
    instructions=instrucciones_orquestador,
    tools=herramientas_orquestador,
    handoffs=handoffs_orquestador,
    model="gpt-4o-mini"
)


In [ ]:
AVAILABLE_TOOLS = {
    "seleccionar_politica_con_llm": seleccionar_politica_con_llm,
    "buscar_contexto_relevante": buscar_contexto_relevante,
    "registrar_pregunta_mysql": registrar_pregunta_mysql,
    "enviar_email_rrhh": enviar_email_rrhh
}


In [ ]:
def handle_tool_calls(tool_calls):
    """
    Manejador para ejecutar las llamadas a las herramientas solicitadas por el LLM.    
    """
    tool_outputs = []
    
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = AVAILABLE_TOOLS.get(function_name)
        
        if not function_to_call:
            tool_outputs.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": f"Error: La herramienta '{function_name}' no existe.",
            })
            continue

        try:
            function_args = json.loads(tool_call.function.arguments)
            print(f"Ejecutando herramienta: {function_name} con argumentos: {function_args}")
            
            function_response = function_to_call(**function_args)
            
            tool_outputs.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": json.dumps(function_response),
            })
        except Exception as e:
            print(f"Error al ejecutar la herramienta {function_name}: {e}")
            tool_outputs.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": f"Error al ejecutar la herramienta: {e}",
            })
    
    return tool_outputs

In [ ]:

runner = Runner(
    agent=orquestador_agente,
    handle_tool_calls=handle_tool_calls 
)

In [ ]:
def chat_func_gradio(message, history):
    """
    Función que Gradio llamará con cada nuevo mensaje del usuario.
    'history' es gestionado por Gradio, pero el 'runner' gestiona su propio estado.
    """
    print(f"\n--- Nueva Interacción ---")
    print(f"Usuario (Gradio): {message}")
    
    try:
        # Asumimos que runner.run() es síncrono y devuelve la respuesta final en string
        # La librería 'agents' gestiona el historial de la conversación internamente.
        response_message = runner.run(message) 
        
    except Exception as e:
        print(f"Error fatal en runner.run: {e}")
        response_message = f"Ocurrió un error al procesar tu solicitud: {e}"
    
    print(f"Agente (Gradio): {response_message}")
    return response_message

# Lanzar la interfaz de Gradio
print("\nIniciando interfaz de Gradio para pruebas...")
print("Abre la URL en tu navegador.")

iface = gr.ChatInterface(
    fn=chat_func_gradio,
    title="Asistente de RRHH Cramer (Prueba)",
    description="Escribe aquí para probar el agente orquestador antes de conectarlo a WhatsApp.",
    examples=[
        "Hola", 
        "Quiero saber sobre la beca de estudios", 
        "Cómo funciona el mutuo acuerdo?",
        "Qué sabes de las vacaciones?"
    ]
)

if __name__ == "__main__":
    iface.launch()


Iniciando interfaz de Gradio para pruebas...
Abre la URL en tu navegador.


NameError: name 'gr' is not defined